## Machine Translation Project (English to Spanish)

In [1]:
import pathlib
import random
import string
import tensorflow.strings as tf_strings
import tensorflow.data as tf_data
import re
from keras.layers import TextVectorization

import keras
import tensorflow as tf
from keras import layers
import json


2025-11-19 14:26:12.514955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763562372.542717     272 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763562372.550728     272 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

## Verify access to the GPU

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18376233146905048556
xla_global_id: -1
]


2025-11-19 14:26:24.124123: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


## Prepare the data
source :"http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"

In [3]:
import keras, pathlib, zipfile, os

# 1. Download ZIP (don't rely on auto extract)
zip_path = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=False,
)

zip_path = pathlib.Path(zip_path)
data_dir = zip_path.parent

# 2. Manual extract to fixed folder
extract_dir = data_dir / "spa-eng"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(extract_dir)

# 3. Locate spa.txt (recursive search)
possible_files = list(extract_dir.rglob("spa.txt"))
text_file = possible_files[0]

# 4. Read file
with open(text_file, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")[:-1]

# 5. Process pairs
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

print("Loaded pairs:", len(text_pairs))
print("Example:", text_pairs[0])


Loaded pairs: 118964
Example: ('Go.', '[start] Ve. [end]')


In [4]:

random.shuffle(text_pairs)

In [5]:

for i in range(5):
    print(text_pairs[i])

("I can't tell you what an honor this is.", '[start] No puedo expresarte el honor que es éste. [end]')
('Yesterday I heard you talking in your sleep.', '[start] Ayer te escuché hablando mientras dormías. [end]')
("I'll call you up this evening.", '[start] Te llamaré esta noche. [end]')
('Two thirds of the employees of this company are engineers.', '[start] Dos tercios de los empleados de esta compañía son ingenieros. [end]')
('I have every confidence in his ability.', '[start] Yo tengo plena confianza en sus capacidades. [end]')


#### Structure of the Dataset

In [6]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [7]:

# parameters
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

## Vectorize the data

In [8]:
def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# vectorization
eng_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

spa_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)

train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

# FAST VECTORIZATION (done once)
eng_train = eng_vectorization(train_eng_texts)
spa_train = spa_vectorization(train_spa_texts)

val_eng_texts = [pair[0] for pair in val_pairs]
val_spa_texts = [pair[1] for pair in val_pairs]

eng_val = eng_vectorization(val_eng_texts)
spa_val = spa_vectorization(val_spa_texts)

# CREATE FAST DATASETS (NO PYTHON LOOPS, GPU-FRIENDLY)
train_ds = tf_data.Dataset.from_tensor_slices((
    {
        "encoder_inputs": eng_train,
        "decoder_inputs": spa_train[:, :-1],
    },
    spa_train[:, 1:]
)).batch(batch_size).prefetch(tf_data.AUTOTUNE)

val_ds = tf_data.Dataset.from_tensor_slices((
    {
        "encoder_inputs": eng_val,
        "decoder_inputs": spa_val[:, :-1],
    },
    spa_val[:, 1:]
)).batch(batch_size).prefetch(tf_data.AUTOTUNE)



#save the vectorization layers
eng_vectorization_config = eng_vectorization.get_config()
eng_vectorization_config.pop('standardize', None)
eng_vocab = eng_vectorization.get_vocabulary()
with open('eng_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vectorization_config, f)
    
with open('eng_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vocab, f)
    
spa_vectorization_config = spa_vectorization.get_config()
spa_vectorization_config.pop('standardize', None)
spa_vocab = spa_vectorization.get_vocabulary()
with open('spa_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vectorization_config, f)
    
with open('spa_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vocab, f)
    



In [9]:
for inputs,targets in train_ds.take(1):
    print(inputs["encoder_inputs"].shape)
    print(targets.shape)

(64, 20)
(64, 20)


In [10]:
import gc
import tensorflow as tf

gc.collect()
tf.keras.backend.clear_session()


In [11]:
# Creating an Encoder
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
        else:
            padding_mask = None
            
        attention_output = self.attention(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = padding_mask,
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config
    
# Creating a Positional Embedding
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim = vocab_size, output_dim = embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim = sequence_length, output_dim = embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0, limit = length, delta = 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        if mask is not None:
            return tf.not_equal(inputs, 0)
        else:
            return None
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "sequence_length": self.sequence_length,
            "embed_dim": self.embed_dim,
        })
        return config
    
# Creating a Decoder
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, encoder_outputs, mask=None):
        casual_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
            padding_mask = tf.minimum(padding_mask, casual_mask)
        else:
            padding_mask = None
            
        attention_output_1 = self.attention_1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)
        
        attention_output_2 = self.attention_2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask,
        )
        
        out_2 = self.layernorm_2(out_1 + attention_output_2)
        proj_output = self.dense_proj(out_2)
        
        return self.layernorm_3(out_2 + proj_output)
    
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, tf.int32)
        mask = tf.reshape(mask,(1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.convert_to_tensor([1, 1]),
            ],
            axis = 0,
        )
        return tf.tile(mask, mult)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config

In [12]:
# define emmbedding dimensions, latent dimensions, and number of heads
embed_dim = 16
latent_dim = 64
num_heads = 1



#Encoder
encoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

#Decoder
decoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")
encoder_seq_inputs = keras.Input(shape = (None, embed_dim), name = "encoder_seq_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoder_seq_inputs)

x = layers.Dropout(0.5)(x)

decoder_outputs = layers.Dense(vocab_size, activation = "softmax")(x)

decoder = keras.Model([decoder_inputs, encoder_seq_inputs], decoder_outputs, name = "decoder")

# Define the final model
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)

In [13]:
epochs = 5

transformer.summary()

transformer.compile(
    optimizer="adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]
)

transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 16)  │    240,320 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 16)  │      3,280 │ positional_embed… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder             │ (None, None,      │    499,720 │ decoder_inputs[0… │
│ (Functional)        │ 15000)            │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 743,320 (2.84 MB)

 Trainable params: 743,320 (2.84 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 538s 406ms/step - accuracy: 0.6341 - loss: 4.5170 - val_accuracy: 0.7315 - val_loss: 1.7719
Epoch 2/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 566s 410ms/step - accuracy: 0.7185 - loss: 1.9246 - val_accuracy: 0.7559 - val_loss: 1.5426
Epoch 3/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 531s 408ms/step - accuracy: 0.7364 - loss: 1.7235 - val_accuracy: 0.7753 - val_loss: 1.3971
Epoch 4/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 533s 409ms/step - accuracy: 0.7499 - loss: 1.5926 - val_accuracy: 0.7885 - val_loss: 1.2940
Epoch 5/5
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 534s 410ms/step - accuracy: 0.7588 - loss: 1.5028 - val_accuracy: 0.7994 - val_loss: 1.2236


In [17]:
transformer.save("/kaggle/working/transformer_model.keras")

In [18]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = sequence_length

def decode_sentence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    input_sentence = input_sentence.lower()
    input_sentence = input_sentence.translate(str.maketrans('', '', strip_chars))
    translated = decode_sentence(input_sentence)
    print(f"input: {input_sentence}")
    print(f"translated: {translated}")
    print()

input: it was an honor
translated: [start] estaba un [UNK] [end]

input: because of her he lived a miserable life
translated: [start] de su [UNK] [UNK] [UNK] [end]

input: he came back soon
translated: [start] Él se [UNK] de [UNK] [end]

input: some believe in ufos and others do not
translated: [start] un gato [UNK] en la [UNK] de la [UNK] [end]

input: get some sleep
translated: [start] [UNK] la ayuda [end]

